# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [25]:
from pyspark.ml.clustering import KMeans, KMeansModel

In [3]:
#Load the Data
data = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [4]:
#Viewing the data
data.head(5)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08),
 Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58),
 Row(Session_Connection_Time=2.0, Bytes Transferred=228.08, Kali_Trace_Used=1, Servers_Corrupted=2.48, Pages_Corrupted=8.0, Location='Bolivia', WPM_Typing_Speed=70.8),
 Row(Session_Connection_Time=20.0, Bytes Transferred=408.5, Kali_Trace_Used=0, Servers_Corrupted=3.57, Pages_Corrupted=8.0, Location='Iraq', WPM_Typing_Speed=71.28)]

In [5]:
data.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

### Preprocessing the Data

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



As the "Location" column is string, we have to either convert or remove the column in order to vectorize and classify the dataset.
Also, as the hackers pretty surely used VPNs, we can safely disregard the column.

In [9]:
new_data = data.drop('Location')
new_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [10]:
vec_assembler = VectorAssembler(inputCols = new_data.columns, outputCol='features')

In [11]:
final_data = vec_assembler.transform(new_data)

In [12]:
## Scaling the Data

In [13]:
from pyspark.ml.feature import StandardScaler

In [14]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [15]:
scalerModel = scaler.fit(final_data)

In [16]:
## Scaling the columns using the Scaler Model

In [17]:
final_data = scalerModel.transform(final_data)

In [18]:
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|      scaledFeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|[2.20042295307707...|
|                    2.0|           228.08|              1|             2.48|     

### Training and Evaluating the Model

In [38]:
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [52]:
predictions=model.transform(final_data).select('prediction')
predictions.show(80)

+----------+
|prediction|
+----------+
|         0|
|         2|
|         0|
|         0|
|         2|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         0|
|         2|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         0|
|         0|
|         2|
|         0|
|         2|
|         0|
|         2|
|         2|
|         0|
|         2|
|         2|
|         0|
|         2|
|         0|
|         0|
|         2|
|         2|
|         0|
|         2|
|         0|
|         2|
|         2|
|         0|
|         2|
|         0|
|         2|
|         0|
|         0|
|         0|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         2|
|         2|

In [53]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[1.21780112 1.37901802 1.99757683 1.37198977 2.55237797 5.29152222]
[2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]
[1.30217042 1.25830099 0.         1.35793211 2.57251009 5.24230473]


In [54]:
# Evaluate clustering by computing Silhouette score
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

IllegalArgumentException: features does not exist. Available: prediction

#### As I installed spark 3.0, I think I am getting errors such as "calculateCost() is not part of KMeans"..
#### So i couldnot run detailed analysis

In [55]:
predictions.show(100)

+----------+
|prediction|
+----------+
|         0|
|         2|
|         0|
|         0|
|         2|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         0|
|         2|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         0|
|         0|
|         2|
|         0|
|         2|
|         0|
|         2|
|         2|
|         0|
|         2|
|         2|
|         0|
|         2|
|         0|
|         0|
|         2|
|         2|
|         0|
|         2|
|         0|
|         2|
|         2|
|         0|
|         2|
|         0|
|         2|
|         0|
|         0|
|         0|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         2|
|         2|

#### However, looking at the first 100 cluster predictions, we can see that none of the observations belong to CLUSTER no. 1
#### Which means that only 2 people were involved during the hack